In [15]:
import pandas as pd
df1 = pd.read_csv('deco_plant.csv')
df1

,id,url
0,0,https://ohou.se/projects/126917/detail?affect_...
1,1,https://ohou.se/projects/126600/detail?affect_...
2,2,https://ohou.se/projects/123056/detail?affect_...
3,3,https://ohou.se/projects/116945/detail?affect_...
4,4,https://ohou.se/projects/127436/detail?affect_...
...,...,...
4431,4687,https://ohou.se/projects/19671/detail?affect_t...
4432,4688,https://ohou.se/projects/18922/detail?affect_t...
4433,4689,https://ohou.se/projects/20141/detail?affect_t...
4434,4690,https://ohou.se/projects/20628/detail?affect_t...


- 김민지 [0:700]

- 박유정 [700:1400]

- 이소현 [1400:2100]

- 이지혜 [2100:2800]

- 최수연 [2800:3500]

- 송나현 [3500:]

In [18]:
start = 2100
end = 2101

In [19]:
df1 = df1[start:end]
df1

,id,url


In [20]:
df_similar = pd.DataFrame(columns = ['id', 'url', 'photo_id', '상품url'])

In [21]:
# 클릭할 영역 찾기
def find_area_and_elements(xpath, class_name):
    clickable_list = driver.find_element(By.XPATH, xpath)
    # 일단 해당 영역으로 마우스(커서)를 이동시킴
    action = webdriver.ActionChains(driver)
    action.move_to_element(clickable_list).perform()

    items = clickable_list.find_elements(By.CLASS_NAME, 'css-1nqec2d')
    return items

In [22]:
from selenium import webdriver
from bs4 import BeautifulSoup as BS
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By # import 문 추가
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore')

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
# 페이지가 모두 로드될 때까지 10초 기다려주도록 처리 (암시적)
driver.implicitly_wait(10)

for idx in df1.index:
    url = df1['url'].loc[idx]
    driver.get(url)

    html = driver.page_source
    soup = BS(html, 'html.parser')
    contents = soup.find_all(['a', 'div'], ['project-detail-image-block__link', 'bpd-view-heading-block'])

    for i, content in enumerate(contents):
        try:
            text = content.text
        except:
            continue
        else:
            if '3초' in text:
                break


    for a_tag in contents[i + 1:]:
        try:
            link = a_tag.attrs['href']
        except:
            break
        else:
            driver.get('https://ohou.se' + link)
            xpath = '/html/body/div[1]/div/div/div[2]/div[1]/div/div[2]/div[2]'
            class_name = 'css-1nqec2d'
            items = find_area_and_elements(xpath, class_name)
            for i in range(0, len(items)):
                if i == 7:
                    break
                
                items[i].click()
                prd_url = driver.current_url
                
                add_row = pd.DataFrame([{'id' : df1['id'].loc[idx], 
                                         'url' : url,
                                         'photo_id' : link.split('/')[-1],
                                         '상품url' : prd_url}])
                df_similar = df_similar.append(add_row)
                
                driver.back()
                items = find_area_and_elements(xpath, class_name)
    
    if (idx + 1) % 50 == 0:
        df_similar.to_csv(f'df_similar_{start}_{idx}.csv', index = False)
        print(idx)


driver.close()

df_similar.to_csv(f'df_similar_{start}_{end}.csv', index = False)



====== WebDriver manager ======
Current google-chrome version is 107.0.5304
Get LATEST chromedriver version for 107.0.5304 google-chrome
Driver [C:\Users\suyeo\.wdm\drivers\chromedriver\win32\107.0.5304.62\chromedriver.exe] found in cache


In [7]:
df_similar.to_csv(f'df_similar_{start}_{end}.csv', index = False)